In [90]:
from ivisual import *
import numpy as np
from numpy import linalg as LA

import os
import fnmatch
from shutil import copyfile

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
% matplotlib notebook

# check to see if a row 'a' dim(1,m)is in the 'b' array dim(n by m)
def inNd(a, b, assume_unique=False):
    a = np.asarray(a, order='C')
    b = np.asarray(b, order='C')
    a = a.ravel().view((np.str, a.itemsize * b.shape[1]))# 6 elements per row
    b = b.ravel().view((np.str, b.itemsize * b.shape[1]))
    return np.in1d(a, b, assume_unique)

def unique_rows(a):
    a = np.ascontiguousarray(a)
    unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
    return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))

# Check if a row is in a array
#https://stackoverflow.com/questions/14766194/testing-whether-a-numpy-array-contains-a-given-row

import itertools

In [91]:
# Load the Gd and Zr atom position in a unit cell
fpath = r'D:\9_Program files\SpinVert\gdzro_ill\gd_ill_60mk-20k_with_paramag/'
fname = 'gd_zr_pos.txt'# from SpinW

at_pos = np.genfromtxt(os.path.join(fpath,fname)).T # 3 by 32: Gd Zr atom positions
gdpos = at_pos[0:16,:]
zrpos = at_pos[16:32,:]
print gdpos,'\n', zrpos
print np.vstack({tuple(row) for row in at_pos}).shape # returen unique values, remoing the duplicated rows

[[ 0.5   0.5   0.5 ]
 [ 0.5   0.25  0.25]
 [ 0.75  0.    0.25]
 [ 0.75  0.75  0.5 ]
 [ 0.25  0.5   0.25]
 [ 0.    0.75  0.25]
 [ 0.    0.    0.5 ]
 [ 0.25  0.25  0.5 ]
 [ 0.    0.5   0.  ]
 [ 0.    0.25  0.75]
 [ 0.25  0.    0.75]
 [ 0.25  0.75  0.  ]
 [ 0.75  0.5   0.75]
 [ 0.5   0.75  0.75]
 [ 0.5   0.    0.  ]
 [ 0.75  0.25  0.  ]] 
[[ 0.    0.    0.  ]
 [ 0.    0.75  0.75]
 [ 0.25  0.5   0.75]
 [ 0.25  0.25  0.  ]
 [ 0.75  0.    0.75]
 [ 0.5   0.25  0.75]
 [ 0.5   0.5   0.  ]
 [ 0.75  0.75  0.  ]
 [ 0.5   0.    0.5 ]
 [ 0.5   0.75  0.25]
 [ 0.75  0.5   0.25]
 [ 0.75  0.25  0.5 ]
 [ 0.25  0.    0.25]
 [ 0.    0.25  0.25]
 [ 0.    0.5   0.5 ]
 [ 0.25  0.75  0.5 ]]
(32L, 3L)


In [147]:
#　Ｐｌｏｔ　ｔｈｅ　ｕｎｉｔ　ｃｅｌｌ
# scene1 = canvas(title='MagStr', width=500, height=500,x=500,y=500, center=(0,0,0),
#                  background=color.black,exit=False)

# spinl = 0.2; atomr = 0.5; cylr = 0.005
# at_pos = np.subtract(at_pos,np.array([0.5,0.5,0.5]))
# # Plot Gd 
# for i in range(16):
#     pointer = sphere(pos=gdpos[i], color=color.red, radius=atomr*0.1)
#     pointer = label(pos=gdpos[i]+(0,0,0.1), text=str(i%16))
# # Plot Zr
# for i in range(16):
#     pointer = sphere(pos=zrpos[i], color=color.blue, radius=atomr*0.1)
#     pointer = label(pos=zrpos[i]+(0,0,0.1), text=str(i%16))
    
# scene1.lights = []
# scene1.ambient=2

In [92]:
# Ｆｉｎｄ　ｔｈｅ　ｂｏｎｄｓ
# Generate 333 supercell for searching the Gd-Zr bond for Gd in the center unit cell
sc3zr = np.zeros([16*3**3,6]) # cell x,y,z, atom x, y, z
#sc333_r = np.array([16*3**3,3])
idx = 0
for i in [-1,0,1]:
    for j in [-1,0,1]:
        for k in [-1,0,1]:
            for l in np.arange(0,16):
                sc3zr[idx,:] = np.hstack((np.array([i,j,k]),zrpos[l,:]))
                idx = idx + 1
                
print sc3zr

##### Find the Gd-Zr bonds includeing the surroundings
# bonds = np.empty((0,6)) #store the 30 bonds in the unitCell: atom1,atom2,pos[a2]-pos[a1]
# idx = 0
# zrp_sc3 = sc3[:,0:3]+sc3[:,3:6] # Zr atom positions in the 333 sCell
# gd_zr_dist = np.sqrt(2)/4
# for i in np.arange(0,16,1):# iter Gd atoms in the central unit cell in the 333 supercell
#     for ii in np.arange(0,16*3**3,1):# iter Zr atoms in the 333 supercell     
#         dist = LA.norm(zrp_sc3[ii]-gdp[i])
#         if dist==gd_zr_dist:
#Problem here             bonds = np.append(bonds, [np.vstack([np.array([i]).reshape([1,1]), sc3[ii,:3],zrp_sc3[ii]-gdp[i]])],axis=0) 
                                                        # Gdidx,  Zrpos L,  r
#             idx +=1
# print bonds

bonds = [] #store the bons 8 cols: atom1,atom2, Zr pos: cell xyz, site xyz
zrp_sc3 = sc3zr[:,0:3]+sc3zr[:,3:6]# abusolute Zr pos in the super cell
gd_zr_dist = np.sqrt(2)/4 #nearest neighbor distance
for i in np.arange(0,16,1):# iter Gd atoms in the central unit cell of the 333 supercell
    for ii in np.arange(0,16*3**3,1):# iter Zr atoms in the 333 supercell     
        dist = LA.norm(zrp_sc3[ii]-gdp[i])
        if dist==gd_zr_dist:
            bonds.append(np.concatenate([np.array([i,ii%16+16]).T, sc3zr[ii,:]])) # GdIdx, ZrIdx ZrPos L, r

bonds = np.asarray(bonds)
bonds

[[-1.   -1.   -1.    0.    0.    0.  ]
 [-1.   -1.   -1.    0.    0.75  0.75]
 [-1.   -1.   -1.    0.25  0.5   0.75]
 ..., 
 [ 1.    1.    1.    0.    0.25  0.25]
 [ 1.    1.    1.    0.    0.5   0.5 ]
 [ 1.    1.    1.    0.25  0.75  0.5 ]]


array([[  0.  ,  18.  ,   0.  ,   0.  ,   0.  ,   0.25,   0.5 ,   0.75],
       [  0.  ,  21.  ,   0.  ,   0.  ,   0.  ,   0.5 ,   0.25,   0.75],
       [  0.  ,  25.  ,   0.  ,   0.  ,   0.  ,   0.5 ,   0.75,   0.25],
       [  0.  ,  26.  ,   0.  ,   0.  ,   0.  ,   0.75,   0.5 ,   0.25],
       [  0.  ,  27.  ,   0.  ,   0.  ,   0.  ,   0.75,   0.25,   0.5 ],
       [  0.  ,  31.  ,   0.  ,   0.  ,   0.  ,   0.25,   0.75,   0.5 ],
       [  1.  ,  19.  ,   0.  ,   0.  ,   0.  ,   0.25,   0.25,   0.  ],
       [  1.  ,  22.  ,   0.  ,   0.  ,   0.  ,   0.5 ,   0.5 ,   0.  ],
       [  1.  ,  24.  ,   0.  ,   0.  ,   0.  ,   0.5 ,   0.  ,   0.5 ],
       [  1.  ,  26.  ,   0.  ,   0.  ,   0.  ,   0.75,   0.5 ,   0.25],
       [  1.  ,  27.  ,   0.  ,   0.  ,   0.  ,   0.75,   0.25,   0.5 ],
       [  1.  ,  28.  ,   0.  ,   0.  ,   0.  ,   0.25,   0.  ,   0.25],
       [  2.  ,  23.  ,   0.  ,  -1.  ,   0.  ,   0.75,   0.75,   0.  ],
       [  2.  ,  25.  ,   0.  ,  -1.  ,   0.  ,   0

In [94]:
# Ｐｌｏｔ　ｔｈｅ　３３３ｓｕｐｅｒｃｅｌｌ　ａｎｄ　ｔｈｅ　ｂｏｎｄ
# scene1 = canvas(title='MagStr', width=800, height=800,x=500,y=500, center=(0,0,0),
#                  background=color.black,exit=False)

# spinl = 0.2; atomr = 0.5; cylr = 0.005
# #at_pos = np.subtract(at_pos,np.array([0.5,0.5,0.5]))
# # Plot Gd 
# for i in range(16):
#     pointer = sphere(pos=gdpos[i], color=color.red, radius=atomr*0.1)
#     pointer = label(pos=gdpos[i]+(0,0,0.1), text=str(i%16))
# # Plot Zr
# for i in range(16*3**3):
#     pointer = sphere(pos=zrp_sc3[i], color=color.blue, radius=atomr*0.1)
#     #pointer = label(pos=zrp_sc3[i]+(0,0,0.1), text=str(i%16))
# # Plot bonds
# bondsV = bonds[:,2:5]+bonds[:,5:8]  
# for i in xrange(bonds.shape[0]):
#     pointer = cylinder(pos=gdpos[np.int(bonds[i,0])], axis=bondsV[i,:]-gdpos[np.int(bonds[i,0])],
#                        radius=cylr, materials=materials.wood)# Gdpos, Zrpos-Gdpos
    
# scene1.lights = []
# scene1.ambient=2

In [95]:
# Ｇｅｎｅｒａｔｅ　ｔｈｅ　６６６ｓｕｐｅｒｃｅｌｌ　ｗｉｔｈ　ａｎｔｉｓｉｔｅ　ｄｅｆｅｃｔｓ
#it is antsite defects, total nb of atoms is not changed
sc6 = np.zeros([16*6**3,7]) # atIdx0-32, cell index x,y,z, atom index x, y, z
idx = 0
for i in xrange(6):
    for j in xrange(6):
        for k in xrange(6):
            for l in np.arange(0,16):
                sc6[idx,:] = np.hstack((np.array([l,i,j,k]),gdpos[l,:]))
                idx = idx + 1
                
print sc6.shape


# Generate antisite disorder
# Randomly select cell and Gd atom in the super cell
mix = 0.2 # how much defects 20%
idxMoved = [] # store the index (0-3456) of the Gd atoms which is moved
gd_moved = np.empty([0,8])# atomID(0-3456), atomIdx(0-16), cell x y z, atom x y z
zr_occupied = np.empty([0,7]) # idx(16-31),cell x y z, atom x y z
idx = 0# total nb of trys of moving Gd
iidx = 0# unique trys of moving Gd
iiidx = 0# succussed to moved to an unoccupied Zr pos

while iiidx<=int(3456*mix):
    whichAt = np.random.randint(3456, size=1)# which atom to move
    idx = idx + 1
    if ~np.in1d(whichAt,gd_moved[:,0]): # if not been moved before
        iidx = iidx + 1
        whichAtIdx = whichAt%16 # the atom idx 1-16 in a unit cell
        BondsAt = bonds[bonds[:,0]==whichAtIdx,:]#the Zr pos that the Gd atom in the [000] unite cell connects: cell xyz, xyz

        whichB = np.random.randint(6, size=1)# move to which Zr pos
        #print whichB, BondsAt[whichB,1:],sc6[whichAt,:]
        #zrTry = np.ravel(sc6[whichAt,:]+BondsAt[whichB,1:]) # wrong !!!!
        zrCell = np.ravel(sc6[whichAt,1:4]+BondsAt[whichB,2:5]) # move to this cell
        if sum(zrCell==-1)|sum(zrCell==6): # not allow moved out of the super cell
            #print zrCell
            continue
        #zrCell[zrCell==-1]=5; zrCell[zrCell==6]=0 # poriodic boundary
        zrTry = np.hstack((BondsAt[whichB,1].reshape([1,1]), zrCell.reshape([1,3]),
                           BondsAt[whichB,5:8])).ravel() # move to this Zr pos: atIdx0-32,xyz,xyz
        #print 'zrTry',zrTry
        if ~inNd(zrTry,zr_occupied):# if the Zr pos not occupied
            gd_moved = np.append(gd_moved, np.hstack((np.array([whichAt,whichAtIdx]).T, sc6[whichAt,1:])),axis=0) 
                        # record the moved Gd: atomID, atom idx, cell xyz, atom xyz
            zr_occupied= np.append(zr_occupied, [zrTry],axis=0)# record the occupied Zr pos
            sc6[whichAt,:] = zrTry # move
            idxMoved.append(whichAt)
            iiidx = iiidx+1
            

sc6[:,0]=sc6[:,0]+1

#print gd_moved, '\n', zr_occupied
#print sc6[idxMoved,:]
print idx,iidx,iiidx
#print np.asarray(idxMoved)

(3456L, 7L)
915 823 692


In [96]:
# Ｐｌｏｔ　ｔｈｅ　ａｎｔｉｓｉｔｅ　ｄｉｆｅｃｔｓ　ｉｎ　ｓｕｐｅｒｃｅｌｌ　ａｎｄ　ｔｈｅｉｒ　ｂｏｎｄｓ
scene1 = canvas(title='MagStr', width=800, height=800,x=500,y=500, center=(0,0,0),
                 background=color.black,exit=False)

spinl = 0.2; atomr = 0.5; cylr = 0.005
#at_pos = np.subtract(at_pos,np.array([0.5,0.5,0.5]))
# Plot Gd 
gdpos = gd_moved[:,2:5]+gd_moved[:,5:8]-np.array([3,3,3])
for i in xrange(iiidx):
    pointer = sphere(pos=gdpos[i], color=color.red, radius=atomr*0.1)
    #pointer = label(pos=gdpos[i]+(0,0,0.1), text=str(i%16))
# Plot Zr
zrpos = zr_occupied[:,1:4]+zr_occupied[:,4:7]-np.array([3,3,3])
for i in range(iiidx):
    pointer = sphere(pos=zrpos[i], color=color.blue, radius=atomr*0.1)
    #pointer = label(pos=zrp_sc3[i]+(0,0,0.1), text=str(i%16))
# Plot bonds
for i in xrange(iiidx):
    pointer = cylinder(pos=gdpos[i], axis=zrpos[i]-gdpos[i],
                       radius=cylr, materials=materials.wood)# Gdpos, Zrpos-Gdpos

print LA.norm(zrpos-gdpos,axis=1)==np.sqrt(2.0)/4
    
scene1.lights = []
scene1.ambient=2

<IPython.core.display.Javascript object>

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [97]:
# Check unique
a = np.array([[1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 0]])
print np.vstack({tuple(row) for row in a})
#print np.unique(sc6)
print np.vstack({tuple(row) for row in sc6}).shape

[[1 1 1 1 1 0]
 [0 1 1 1 0 0]
 [1 1 1 0 0 0]]
(3456L, 7L)


In [98]:
# Save
path = r'D:\9_Program files\SpinVert\gdzro_ill\gd_ill_60mk-20k_with_paramag\0a.txt'

# first save site
names  = np.repeat('SITE',32)
allnn = np.column_stack([names,at_pos]) #nb = np.char.mod("%4.2f", at_pos)
#print allnn
np.savetxt(path,allnn, fmt="%s") # workings

# second save super lattice with nearest neighbor antisite disorder 
f = open(path, 'a') # append
names  = np.repeat('SPIN',3456).astype('str')
outputs = map(lambda x: "%s %i %i %i %i %f %f %f " % (x[0],x[1][0],x[1][1],x[1][2],x[1][3],x[1][4],x[1][5],x[1][6]),
              zip(names, sc6)) 
data = '\n'.join(outputs) # Concat all strings in list, separated by line break
f.write(data)
f.close()



# working but TXT can not see \n only notepad can but Spinvert can not read correctly
# f = open(path, 'wb')
# for i in range(32):
#     f.write("%s %5.2f %5.2f %5.2f \n" % ('SITE',at_pos[i,0], at_pos[i,1],at_pos[i,2]))
# for item in sc6:
#     f.write("%s %i %i %i %i %5.2f %5.2f %5.2f \n" % ('SPIN',item[0],item[1],item[2],item[3],
#                                                               item[4],item[5],item[6]))
# f.close()

# working but writing zero as 0. not 0.0
# f = open(path, 'w')
# for i in gdpos:
#     f.write('SITE %s\n'%str(i)[1:-1]);# 1:-1 for removing brackests 
# f.close()  
# np.savetxt(path, zip(names,at_pos[:,0],at_pos[:,1],at_pos[:,2]), fmt="%s, %5.2f, %5.2f, %5.2f")# %s problem

# dtype of 'names' is not compatable with %s
# names  = np.repeat('SITE',32)
# with open(path,'wb') as f:
#     np.savetxt(f,zip(names,at_pos[:,0],at_pos[:,1],at_pos[:,2]), fmt="%5s %10.3f %10.3f %10.3f")

# names  = np.repeat('SPIN',3456)
# allnn = np.column_stack([names,sc6])
# with open(path,'a') as f_handle: # append
#     np.savetxt(f_handle,allnn,fmt='%s %i %f %f %f %f %f %f') #%s problem

# not working
# ab = np.zeros([32,], dtype=[('var1', 'S6'), ('', float),('', float),('', float)])
# ab['var1'] = names
# ab[:] = at_pos
# np.savetxt(path, ab, fmt="%10s %10.3f")


In [99]:
# Process the output of Spinvet: fill 'spins.txt' to be including 32 sites with spin zero at the empty sites
# For spindiff calcuation
fpath = r'D:\9_Program files\SpinVert\0_spinvert_program\spinvert_with_modifed\programs\1st_supercell_defects'
fname = '10k_0p994_s3_wl2p41_p3500i_spins_*.txt'

# Build the supercell&spins: atIdx(1-32), cell xyz, spin xyz
atIdxes = np.repeat([np.arange(1,33)],6**3,axis=0).ravel().reshape([-1,1])#1,2,3...31,32, 1,2,3....

scIdxes=list(itertools.product([0,1,2,3,4,5], repeat=3))
scIdxes = np.repeat(scIdxes,32, axis=0).reshape([-1,3])#000,000,.... 001,001

new_s = np.hstack((  atIdxes,scIdxes,np.zeros([32*6**3,3]) ))
names  = np.repeat('SPIN',32*6**3).astype('str')

ii=0 #index for different files
idx = 0
for file in os.listdir(fpath):
    if fnmatch.fnmatch(file, fname):      
        print(file)
        filepath = os.path.join(fpath,file)
        out_s = np.genfromtxt(filepath, skip_header=44,usecols=[1,2,3,4,5,6,7]) # 3 by 32: Gd Zr atom positions
        idx = 0
        for item in out_s:
            matching  = np.where( np.prod(new_s[:,0:4]==item[0:4],axis=1) )#find the idx
            new_s[matching,4:7] = item[4:7]#copy the spins
            idx = idx + 1
        print idx, 'spins copied'
        
        # save: copy the headers and then append the filled supercell + spins                   
        outputs = map(lambda x: "%s %i %i %i %i %f %f %f " % (x[0],x[1][0],x[1][1],x[1][2],x[1][3],x[1][4],x[1][5],x[1][6]),                                                            zip(names, new_s)) 
        data = '\n'.join(outputs) # Concat all strings in list, separated by line break
        
        newpath =filepath.replace('spins','filled_spins')
        with open(filepath,'r') as infile, open(newpath, 'w') as outfile:
            for line in infile:
                if line.rstrip('\n').split()[0] != "SPIN": # the line not starting with 'spins'
                    outfile.write(line)# copy the heading to the file
                else:
                    break    
            outfile.write(data)#append the 'spin'

#print new_s.shape        
        

10k_0p994_s3_wl2p41_p3500i_spins_01.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_02.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_03.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_04.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_05.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_06.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_07.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_08.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_09.txt
3456 spins copied
10k_0p994_s3_wl2p41_p3500i_spins_10.txt
3456 spins copied


In [67]:
# change some file names
from shutil import copyfile
fpath = r'D:\9_Program files\SpinVert\0_spinvert_program\spinvert_with_modifed\programs\old_fit/'
fname = '10k_0p994_s3_wl2p41_p3500i_original_spins_*.txt'
for file in os.listdir(fpath):
    if fnmatch.fnmatch(file, fname):      
        print(file)
        filepath = os.path.join(fpath,file)
        newpath =filepath.replace('original_spins','spins')
        copyfile(filepath,newpath)# copy the origal file


10k_0p994_s3_wl2p41_p3500i_original_spins_01.txt
10k_0p994_s3_wl2p41_p3500i_original_spins_02.txt
10k_0p994_s3_wl2p41_p3500i_original_spins_03.txt
10k_0p994_s3_wl2p41_p3500i_original_spins_04.txt
10k_0p994_s3_wl2p41_p3500i_original_spins_05.txt


In [32]:
import itertools
permu=list(itertools.product([0,1,2], repeat=3))
#permu=np.array(permu)
np.repeat(permu,6, axis=0)
#print permu
print np.repeat([np.arange(1,33)],3,axis=0).ravel()
np.prod([ True , True,  True,  False])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18
 19 20 21 22 23 24 25 26 27 28 29 30 31 32  1  2  3  4  5  6  7  8  9 10 11
 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32]


0

In [98]:
k = np.array([[1,2,3,4],[3,4,3,5],[5,6,3,4]])
k2 = np.array([1,2,3,4])
#np.sum(a==b)
print k2.view(dtype='f,i,i,i')
print k.view(dtype='i,i,i,i')
np.in1d(k2.view(dtype='i,i,i,i'),k.view(dtype='i,i,i,i'))

[(1.401298464324817e-45, 2, 3, 4)]
[[(1, 2, 3, 4)]
 [(3, 4, 3, 5)]
 [(5, 6, 3, 4)]]


array([ True], dtype=bool)

In [ ]:
np.in1d(k2.view(dtype='i,i').reshape(k2.shape[0]),k.view(dtype='i,i').reshape(k.shape[0]))
